**Imports**

To rebuild the database completely from scratch:
1. Delete "db.sqlite3" file
2. Delete migrations files
3. Run: `python manage.py makemigrations`
4. Run: `python manage.py migrate`
5. Run: The following cell

In [2]:
import os

if os.getcwd().endswith('sqlalchemy_utils'):
    os.chdir('..')

from sqlalchemy_utils.db_session import get_session
from sqlalchemy_utils import db_utils
from sqlalchemy_utils.models_sqlalchemy_orm import Base, GroupLevelMapFile
from sqlalchemy import text

session = get_session()

Base.metadata.create_all(session.get_bind())

db_utils.insert_default_users('sqlalchemy_utils/data/default_users.json', session)
db_utils.insert_causes_from_json('sqlalchemy_utils/data/causes.json', session, override_existing=True)
db_utils.insert_domains_from_json('sqlalchemy_utils/data/domains.json', session)
db_utils.insert_symptoms_from_json('sqlalchemy_utils/data/symptoms.json', session)
db_utils.insert_coordinate_spaces_from_json('sqlalchemy_utils/data/coordinate_spaces.json', session)
db_utils.insert_sexes_from_json('sqlalchemy_utils/data/sexes.json', session, override_existing=True)
db_utils.insert_handedness_from_json('sqlalchemy_utils/data/handedness.json', session)
db_utils.insert_connectomes_from_json('sqlalchemy_utils/data/connectomes.json', session)
db_utils.insert_statistic_types_from_json('sqlalchemy_utils/data/statistic_types.json', session)
db_utils.insert_dimensions_from_json('sqlalchemy_utils/data/dimensions.json', session)
db_utils.insert_modalities_from_json('sqlalchemy_utils/data/imaging_modalities.json', session)
db_utils.process_parcellation(db_utils.fetch_atlas_3209c91v(), session)
db_utils.insert_map_types_from_json('sqlalchemy_utils/data/map_types.json', session)
db_utils.insert_levels_from_json('sqlalchemy_utils/data/traininglevels/level_config.json', session)

session.close()

User with username 'josephturner' or email 'jiturner@bwh.harvard.edu' already exists. Skipping.
User with username 'mikefox' or email 'mdfox@bwh.harvard.edu' already exists. Skipping.
User with username 'michaelferguson' or email 'mferguson7@bwh.harvard.edu' already exists. Skipping.
User with username 'fredschaper' or email 'fschaper@bwh.harvard.edu' already exists. Skipping.
User with username 'jarednielsen' or email 'jarednielsen@byu.edu' already exists. Skipping.
User with username 'mengyuanding' or email 'mding3@bwh.harvard.edu' already exists. Skipping.
There are now 9 causes in the database.
There are now 6 domains in the database.
There are now 32 subdomains in the database.
There are now 100 symptoms in the database.
There are now 449 synonyms in the database.
There are now 108 mesh terms in the database.
There are now 3 coordinate spaces in the database.
There are now 3 sexes in the database.
There are now 4 handedness records in the database.
There are now 7 connectomes in t

/Users/jt041/miniforge3/envs/django_env/lib/python3.12/site-packages/nilearn/maskers/nifti_masker.py:110: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(
/Users/jt041/miniforge3/envs/django_env/lib/python3.12/site-packages/joblib/memory.py:312: UserWarning: Casting data from int32 to float32
  return self.func(*args, **kwargs)


MapType 'Sensitivity Map (Overlap)' already exists. Skipping.
MapType 'Average Map' already exists. Skipping.
MapType 'Specificity Map' already exists. Skipping.
There are now 3 MapTypes in the database.
✅ Uploaded /Users/jt041/repos/lesionbank_new/sqlalchemy_utils/data/traininglevels/01/screenshot.png to S3 bucket 'lesionbucket' with key 'traininglevels/01/screenshot.png'.
✅ Uploaded /Users/jt041/repos/lesionbank_new/sqlalchemy_utils/data/traininglevels/01/Baird_A.nii.gz to S3 bucket 'lesionbucket' with key 'traininglevels/01/Baird_A.nii.gz'.
✅ Uploaded /Users/jt041/repos/lesionbank_new/sqlalchemy_utils/data/traininglevels/02/screenshot.png to S3 bucket 'lesionbucket' with key 'traininglevels/02/screenshot.png'.
✅ Uploaded /Users/jt041/repos/lesionbank_new/sqlalchemy_utils/data/traininglevels/02/miller_A.nii.gz to S3 bucket 'lesionbucket' with key 'traininglevels/02/miller_A.nii.gz'.
✅ Uploaded /Users/jt041/repos/lesionbank_new/sqlalchemy_utils/data/traininglevels/03/screenshot.png to

In [1]:
# manage_levels.py

import os

# Adjust working directory if necessary
if os.getcwd().endswith('sqlalchemy_utils'):
    os.chdir('..')

from sqlalchemy_utils.db_session import get_engine
from sqlalchemy_utils.models_sqlalchemy_orm import Base, Level, UserLevelProgress
from sqlalchemy import inspect

def drop_tables(engine):
    inspector = inspect(engine)
    with engine.connect() as connection:
        trans = connection.begin()
        try:
            # Drop 'user_level_progress' table if it exists
            if 'user_level_progress' in inspector.get_table_names():
                UserLevelProgress.__table__.drop(engine)
                print("✅ Dropped 'user_level_progress' table.")
            else:
                print("ℹ️  'user_level_progress' table does not exist. No need to drop.")

            # Drop 'levels' table if it exists
            if 'levels' in inspector.get_table_names():
                Level.__table__.drop(engine)
                print("✅ Dropped 'levels' table.")
            else:
                print("ℹ️  'levels' table does not exist. No need to drop.")

            # Commit the transaction
            trans.commit()
        except Exception as e:
            trans.rollback()
            print(f"❌ Failed to drop tables: {e}")

def create_tables(engine):
    try:
        Base.metadata.create_all(engine, tables=[
            Level.__table__,
            UserLevelProgress.__table__
        ])
        print("✅ Recreated tables with updated schema.")
    except Exception as e:
        print(f"❌ Failed to create tables: {e}")

engine = get_engine()

drop_tables(engine)
create_tables(engine)

✅ Dropped 'user_level_progress' table.
✅ Dropped 'levels' table.
✅ Recreated tables with updated schema.


**Update a table (say you have a new column in the table)**

In [4]:
# add_internal_use_only_column.py

import os

# Adjust working directory if necessary
if os.getcwd().endswith('sqlalchemy_utils'):
    os.chdir('..')

from sqlalchemy_utils.db_session import get_engine
from sqlalchemy import inspect, Column, Boolean, text
from sqlalchemy.exc import OperationalError

def add_internal_use_only_column(engine, table_name):
    inspector = inspect(engine)
    columns = inspector.get_columns(table_name)
    column_names = [column['name'] for column in columns]
    if 'internal_use_only' not in column_names:
        # Column doesn't exist, add it
        alter_stmt = f'ALTER TABLE {table_name} ADD COLUMN internal_use_only BOOLEAN DEFAULT FALSE'
        alter_stmt = text(alter_stmt)
        with engine.connect() as conn:
            trans = conn.begin()
            try:
                conn.execute(alter_stmt)
                trans.commit()
                print(f"✅ Added 'internal_use_only' column to '{table_name}' table.")
            except OperationalError as e:
                trans.rollback()
                print(f"❌ Failed to add 'internal_use_only' column to '{table_name}' table: {e}")
    else:
        print(f"ℹ️  'internal_use_only' column already exists in '{table_name}' table. Dropping and recreating it.")
        alter_stmt = f'ALTER TABLE {table_name} DROP COLUMN internal_use_only'
        alter_stmt = text(alter_stmt)
        with engine.connect() as conn:
            trans = conn.begin()
            try:
                conn.execute(alter_stmt)
                trans.commit()
                print(f"✅ Dropped 'internal_use_only' column from '{table_name}' table.")
            except OperationalError as e:
                trans.rollback()
                print(f"❌ Failed to drop 'internal_use_only' column from '{table_name}' table: {e}")
        add_internal_use_only_column(engine, table_name)

engine = get_engine()

add_internal_use_only_column(engine, 'subjects')
add_internal_use_only_column(engine, 'symptoms')

ℹ️  'internal_use_only' column already exists in 'subjects' table. Dropping and recreating it.
✅ Dropped 'internal_use_only' column from 'subjects' table.
✅ Added 'internal_use_only' column to 'subjects' table.
ℹ️  'internal_use_only' column already exists in 'symptoms' table. Dropping and recreating it.
✅ Dropped 'internal_use_only' column from 'symptoms' table.
✅ Added 'internal_use_only' column to 'symptoms' table.
